In [1]:
import tensorflow as tf
import numpy as np
import os
 
def get_files(path_pos,path_neg,label_pos,label_neg):
    TC = []
    label_TC = []
    nonTC = []
    label_nonTC = []
    # data loader
    file_dir_TC=path_pos
    file_dir_nonTC=path_neg
    for file in os.listdir(file_dir_TC):
        name = file.split('_')
        if name[0] == label_pos:
            TC.append(file_dir_TC + file)
            label_TC.append(1)
    for file in os.listdir(file_dir_nonTC):
        name = file.split('_')
        if name[0] == label_neg:
            nonTC.append(file_dir_nonTC + file)
            label_nonTC.append(0)
    print("There are %d TC\nThere are %d nonTC" % (len(TC), len(nonTC)))
 
    # shuffle
    image_list = np.hstack((TC, nonTC))
    label_list = np.hstack((label_TC, label_nonTC))
    temp = np.array([image_list, label_list])
    temp = temp.transpose()    
    np.random.shuffle(temp)
 
    image_list = list(temp[:, 0])
    label_list = list(temp[:, 1])
    label_list = [int(i) for i in label_list]
 
    return image_list, label_list
 
# img_list,label_list = get_files(file_dir)
 
# batch
def get_batch(image, label, image_W, image_H, batch_size, capacity):   
    image = tf.cast(image, tf.string)
    label = tf.cast(label, tf.int32)
 
    # queue
    input_queue = tf.train.slice_input_producer([image, label])
 
    image_contents = tf.read_file(input_queue[0])
    label = input_queue[1]
    image = tf.image.decode_jpeg(image_contents, channels=1)
 
    # resize
    image = tf.image.resize_images(image, [image_H, image_W], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    image = tf.cast(image, tf.float32)
    # image = tf.image.per_image_standardization(image)  
    image_batch, label_batch = tf.train.batch([image, label],
                                              batch_size=batch_size,
                                              num_threads=64,  
                                              capacity=capacity)
  
    return image_batch, label_batch

In [2]:
from tensorflow.contrib import slim
def AlexNet(x_input):
    with slim.arg_scope([slim.conv2d, slim.fully_connected], 
                        activation_fn=tf.nn.relu, 
                        weights_initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.01), 
                        weights_regularizer=slim.l2_regularizer(0.0005)):
        # layer 1
        net = slim.conv2d(x_input, 96, [3,3], padding="VALID", stride=4, scope="conv_1")
        net = tf.nn.local_response_normalization(net, depth_radius=5, bias=2, alpha=0.0001, beta=0.75)
        net = slim.max_pool2d(net, [3,3], 2, scope="pool_1")
        # layer 2
        net = slim.conv2d(net, 256, [3,3], scope="conv_2")
        net = tf.nn.local_response_normalization(net, depth_radius=5, bias=2, alpha=0.0001, beta=0.75)
        net = slim.max_pool2d(net, [3,3], 2, scope="pool_2")
        # layer 3 4
        net = slim.repeat(net, 2, slim.conv2d, 384, [3,3], scope="conv_3_4")
        # layer 5
        net = slim.conv2d(net, 256, [3,3], scope="conv_5")
        net = slim.max_pool2d(net, [3,3], 2, scope="pool_5")
        # flatten
        net = slim.flatten(net, scope="flatten")
        # layer 6
        net = slim.fully_connected(net, 4096, scope="fc_6")
        net = slim.dropout(net, keep_prob=0.5)
        # layer 7
        net = slim.fully_connected(net, 4096, scope="fc_7")
        net = slim.dropout(net, keep_prob=0.5)
        #output
        net = slim.fully_connected(net, 2, scope="output", activation_fn=None)

        return net

def losses(logits, labels, name):
    with tf.variable_scope('loss') as scope:
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits \
            (logits=logits, labels=labels, name='xentropy_per_example')
        loss = tf.reduce_mean(cross_entropy, name='loss')
        if name=='train':
            tf.summary.scalar(scope.name + '/train_loss', loss)
        if name=='valid':
            tf.summary.scalar(scope.name + '/valid_loss', loss)
    return loss
 
def trainning(loss, learning_rate):
    with tf.name_scope('optimizer'):
        optimizer = tf.train.AdamOptimizer(learning_rate= learning_rate)
        global_step = tf.Variable(0, name='global_step', trainable=False)
        train_op = optimizer.minimize(loss, global_step= global_step)
    return train_op
 
def evaluation(logits, labels, name):
    with tf.variable_scope('accuracy') as scope:
        correct = tf.nn.in_top_k(logits, labels, 1)
        correct = tf.cast(correct, tf.float16)
        accuracy = tf.reduce_mean(correct)
        if name=='train':
            tf.summary.scalar(scope.name + '/train_accuracy', accuracy)
        if name=='valid':
            tf.summary.scalar(scope.name + '/valid_accuracy', accuracy)
    return accuracy

def recall_precision(logits, labels, name):
    logits = tf.cast(logits, tf.int64)
    labels = tf.cast(labels, tf.int64)
    predict = tf.arg_max(logits,1)
    with tf.variable_scope('recall_precision') as scope:
        TP = tf.count_nonzero(predict * labels)
        TN = tf.count_nonzero((predict - 1) * (labels - 1))
        FN = tf.count_nonzero((predict - 1) * labels)
        FP = tf.count_nonzero(predict * (labels - 1))
        precision = tf.divide(TP, TP + FP)
        recall = tf.divide(TP, TP + FN)
        precision = tf.cast(precision, dtype=tf.float64)
        recall = tf.cast(recall, dtype=tf.float64)
        #f1 = 2 * precision * recall / (precision + recall)
        #f1 = tf.cast(f1, dtype=tf.float32)
        if name=='train':
            tf.summary.scalar(scope.name + '/train_precision', precision)
            tf.summary.scalar(scope.name + '/train_recall', recall)
        if name=='valid':
            tf.summary.scalar(scope.name + '/valid_precision', precision)
            tf.summary.scalar(scope.name + '/valid_recall', recall)
    return precision, recall

In [ ]:
import os
import numpy as np
import tensorflow as tf
tf.reset_default_graph()
N_CLASSES = 2 
IMG_W = 224  # resize
IMG_H = 224
BATCH_SIZE = 64
CAPACITY = 2000
MAX_STEP = 5000000 
learning_rate = 0.0001 
 
train_dir = './train_alexnet/'
logs_train_dir = './train_alexnet/'

file_dir_TC='/home/ubuntu/data/TC/'
file_dir_nonTC='/home/ubuntu/data/nonTC/'
# file_dir_valTC='/home/ubuntu/data/valTC/'
# file_dir_valnonTC='/home/ubuntu/data/valnonTC/'

train, train_label = get_files(file_dir_TC,file_dir_nonTC,'TC','nonTC')
# valid, valid_label = get_files(file_dir_valTC,file_dir_valnonTC,'valTC','valnonTC')
train_batch,train_label_batch=get_batch(train,
                                train_label,
                                IMG_W,
                                IMG_H,
                                BATCH_SIZE,
                                CAPACITY)

# valid_batch,valid_label_batch=get_batch(valid,
#                                 valid_label,
#                                 IMG_W,
#                                 IMG_H,
#                                 BATCH_SIZE,
#                                 CAPACITY)
# x = tf.placeholder(tf.float32, [BATCH_SIZE,IMG_W,IMG_H,1])
# y = tf.placeholder(tf.float32, [BATCH_SIZE])
# y = tf.cast(y,tf.int64)

train_logits = AlexNet(train_batch)
train_loss = losses(train_logits, train_label_batch, 'train')
# valid_loss = losses(train_logits, y, 'valid')
train_op = trainning(train_loss, learning_rate)
train__acc = evaluation(train_logits, train_label_batch, 'train')
# valid__acc = evaluation(train_logits, y, 'valid')
train_recall, train_precision = recall_precision(train_logits, train_label_batch, 'train')
# val_recall, val_precision = recall_precision(train_logits, y, 'valid')
summary_op = tf.summary.merge_all() 
 
sess = tf.Session()

train_writer = tf.summary.FileWriter(logs_train_dir, sess.graph)

saver = tf.train.Saver()

sess.run(tf.global_variables_initializer())
 

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)
 
try:
    print('----------training start---------')
    for step in np.arange(MAX_STEP):
        
        if coord.should_stop():
                break
       
        _, tra_loss, tra_acc, tra_recall, tra_precision= sess.run([train_op, train_loss, train__acc, train_recall, train_precision])
        
        if step % 50 == 0:
            print('Step %d, train loss = %.2f, train accuracy = %.2f%%, train recall = %.2f%%, train precision = %.2f%%' %(step, tra_loss, tra_acc*100.0, tra_recall*100.0, tra_precision*100.0))
            
#         if step % 200 == 0:    
#             val_loss, val_acc, cal_recall, val_precision = sess.run([valid_loss, valid__acc, val_recall, val_precision], feed_dict={x:valid_batch,y:valid_label_batch})
#             print('Step %d, valid loss = %.2f, valid accuracy = %.2f%%, valid recall = %.2f%%, valid precision = %.2f%%' %(step, val_loss, val_acc*100.0, val_recall*100.0, val_precision*100.0))
            summary_str = sess.run(summary_op)
            train_writer.add_summary(summary_str, step)
      
        if step % 2000 == 0 or (step + 1) == MAX_STEP:
            checkpoint_path = os.path.join(logs_train_dir, 'model.ckpt')
            saver.save(sess, checkpoint_path, global_step=step)

except tf.errors.OutOfRangeError:
    print('Done training -- epoch limit reached')

finally:
    coord.request_stop()

There are 2016816 TC
There are 1737956 nonTC
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Use `argmax` instead
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
----------training start---------
Step 0, train loss = 0.69, train accuracy = 43.75%, train recall = nan%, train precision = 0.00%
Step 50, train loss = 0.52, train accuracy = 71.88%, train recall = 94.44%, train precision = 45.95%
Step 100, train loss = 0.55, train accuracy = 76.56%, train recall = 88.24%, train precision = 50.00%
Step 150, train loss = 0.54, train accuracy = 67.19%, train recall = 100.00%, train precision = 15.79%
Step 200, train loss = 0.46, train accuracy = 81.25%, train recall = 84.00%, train precision = 58.33%
Step 250, train loss = 0.33, train accuracy = 89.06%, train recall = 100.00%, train precision = 51.35%
Step 300, train 

Step 3400, train loss = 0.29, train accuracy = 85.94%, train recall = 100.00%, train precision = 64.86%
Step 3450, train loss = 0.30, train accuracy = 89.06%, train recall = 94.74%, train precision = 58.06%
Step 3500, train loss = 0.32, train accuracy = 82.81%, train recall = 100.00%, train precision = 55.56%
Step 3550, train loss = 0.30, train accuracy = 85.94%, train recall = 95.83%, train precision = 69.70%
Step 3600, train loss = 0.27, train accuracy = 89.06%, train recall = 100.00%, train precision = 54.29%
Step 3650, train loss = 0.30, train accuracy = 84.38%, train recall = 96.15%, train precision = 67.57%
Step 3700, train loss = 0.37, train accuracy = 89.06%, train recall = 88.24%, train precision = 41.67%
Step 3750, train loss = 0.49, train accuracy = 73.44%, train recall = 94.12%, train precision = 47.06%
Step 3800, train loss = 0.27, train accuracy = 92.19%, train recall = 100.00%, train precision = 57.14%
Step 3850, train loss = 0.22, train accuracy = 90.62%, train recall =

Step 7400, train loss = 0.24, train accuracy = 89.06%, train recall = 100.00%, train precision = 71.43%
Step 7450, train loss = 0.33, train accuracy = 82.81%, train recall = 94.12%, train precision = 45.71%
Step 7500, train loss = 0.18, train accuracy = 93.75%, train recall = 100.00%, train precision = 67.57%
Step 7550, train loss = 0.27, train accuracy = 87.50%, train recall = 94.12%, train precision = 61.54%
Step 7600, train loss = 0.28, train accuracy = 89.06%, train recall = 96.55%, train precision = 68.29%
Step 7650, train loss = 0.31, train accuracy = 89.06%, train recall = 89.29%, train precision = 67.57%
Step 7700, train loss = 0.42, train accuracy = 87.50%, train recall = 93.10%, train precision = 79.41%
Step 7750, train loss = 0.21, train accuracy = 90.62%, train recall = 96.30%, train precision = 86.67%
Step 7800, train loss = 0.34, train accuracy = 87.50%, train recall = 95.45%, train precision = 65.62%
Step 7850, train loss = 0.33, train accuracy = 89.06%, train recall = 9

Step 11350, train loss = 0.35, train accuracy = 84.38%, train recall = 95.24%, train precision = 66.67%
Step 11400, train loss = 0.24, train accuracy = 89.06%, train recall = 96.30%, train precision = 76.47%
Step 11450, train loss = 0.18, train accuracy = 92.19%, train recall = 100.00%, train precision = 78.05%
Step 11500, train loss = 0.29, train accuracy = 87.50%, train recall = 100.00%, train precision = 56.25%
Step 11550, train loss = 0.32, train accuracy = 89.06%, train recall = 96.00%, train precision = 75.00%
Step 11600, train loss = 0.32, train accuracy = 85.94%, train recall = 96.77%, train precision = 83.33%
Step 11650, train loss = 0.27, train accuracy = 87.50%, train recall = 100.00%, train precision = 68.42%
Step 11700, train loss = 0.41, train accuracy = 82.81%, train recall = 95.45%, train precision = 60.00%
Step 11750, train loss = 0.24, train accuracy = 85.94%, train recall = 100.00%, train precision = 56.76%
Step 11800, train loss = 0.16, train accuracy = 93.75%, trai

Step 15300, train loss = 0.27, train accuracy = 87.50%, train recall = 100.00%, train precision = 67.57%
Step 15350, train loss = 0.22, train accuracy = 85.94%, train recall = 100.00%, train precision = 68.57%
Step 15400, train loss = 0.21, train accuracy = 90.62%, train recall = 95.45%, train precision = 65.62%
Step 15450, train loss = 0.15, train accuracy = 98.44%, train recall = 100.00%, train precision = 59.38%
Step 15500, train loss = 0.19, train accuracy = 93.75%, train recall = 100.00%, train precision = 82.93%
Step 15550, train loss = 0.26, train accuracy = 89.06%, train recall = 100.00%, train precision = 56.25%
Step 15600, train loss = 0.14, train accuracy = 95.31%, train recall = 100.00%, train precision = 72.22%
Step 15650, train loss = 0.32, train accuracy = 85.94%, train recall = 100.00%, train precision = 52.63%
Step 15700, train loss = 0.28, train accuracy = 89.06%, train recall = 100.00%, train precision = 50.00%
Step 15750, train loss = 0.22, train accuracy = 90.62%, 

...

Step 86400, train loss = 0.08, train accuracy = 95.31%, train recall = 100.00%, train precision = 87.23%
Step 86450, train loss = 0.08, train accuracy = 96.88%, train recall = 100.00%, train precision = 85.00%
Step 86500, train loss = 0.05, train accuracy = 98.44%, train recall = 100.00%, train precision = 92.50%
Step 86550, train loss = 0.15, train accuracy = 96.88%, train recall = 100.00%, train precision = 80.00%
Step 86600, train loss = 0.14, train accuracy = 93.75%, train recall = 100.00%, train precision = 82.14%
Step 86650, train loss = 0.15, train accuracy = 92.19%, train recall = 96.55%, train precision = 84.85%
Step 86700, train loss = 0.21, train accuracy = 95.31%, train recall = 93.94%, train precision = 79.49%
Step 86750, train loss = 0.19, train accuracy = 95.31%, train recall = 100.00%, train precision = 71.05%
Step 86800, train loss = 0.27, train accuracy = 89.06%, train recall = 96.88%, train precision = 83.78%
Step 86850, train loss = 0.16, train accuracy = 92.19%, tr

Step 90350, train loss = 0.15, train accuracy = 93.75%, train recall = 100.00%, train precision = 91.67%
Step 90400, train loss = 0.31, train accuracy = 93.75%, train recall = 94.12%, train precision = 80.00%
Step 90450, train loss = 0.10, train accuracy = 95.31%, train recall = 96.97%, train precision = 94.12%
Step 90500, train loss = 0.09, train accuracy = 95.31%, train recall = 100.00%, train precision = 86.49%
Step 90550, train loss = 0.22, train accuracy = 92.19%, train recall = 96.43%, train precision = 81.82%
Step 90600, train loss = 0.19, train accuracy = 95.31%, train recall = 100.00%, train precision = 81.08%
Step 90650, train loss = 0.23, train accuracy = 90.62%, train recall = 92.59%, train precision = 83.33%
Step 90700, train loss = 0.15, train accuracy = 93.75%, train recall = 96.77%, train precision = 83.33%
Step 90750, train loss = 0.25, train accuracy = 89.06%, train recall = 96.15%, train precision = 75.76%
Step 90800, train loss = 0.10, train accuracy = 96.88%, train

Step 94300, train loss = 0.23, train accuracy = 90.62%, train recall = 96.43%, train precision = 81.82%
Step 94350, train loss = 0.11, train accuracy = 95.31%, train recall = 100.00%, train precision = 80.56%
Step 94400, train loss = 0.20, train accuracy = 89.06%, train recall = 94.74%, train precision = 72.00%
Step 94450, train loss = 0.16, train accuracy = 92.19%, train recall = 100.00%, train precision = 80.65%
Step 94500, train loss = 0.16, train accuracy = 92.19%, train recall = 100.00%, train precision = 76.32%
Step 94550, train loss = 0.26, train accuracy = 90.62%, train recall = 93.10%, train precision = 72.97%
Step 94600, train loss = 0.16, train accuracy = 90.62%, train recall = 100.00%, train precision = 80.56%
Step 94650, train loss = 0.17, train accuracy = 95.31%, train recall = 96.97%, train precision = 80.00%
Step 94700, train loss = 0.26, train accuracy = 89.06%, train recall = 100.00%, train precision = 75.68%
Step 94750, train loss = 0.29, train accuracy = 82.81%, tra

Step 98250, train loss = 0.17, train accuracy = 95.31%, train recall = 100.00%, train precision = 76.47%
Step 98300, train loss = 0.17, train accuracy = 96.88%, train recall = 100.00%, train precision = 90.62%
Step 98350, train loss = 0.13, train accuracy = 95.31%, train recall = 100.00%, train precision = 81.58%
Step 98400, train loss = 0.15, train accuracy = 95.31%, train recall = 100.00%, train precision = 83.78%
Step 98450, train loss = 0.10, train accuracy = 96.88%, train recall = 100.00%, train precision = 84.38%
Step 98500, train loss = 0.11, train accuracy = 96.88%, train recall = 100.00%, train precision = 86.67%
Step 98550, train loss = 0.11, train accuracy = 93.75%, train recall = 100.00%, train precision = 90.24%
Step 98600, train loss = 0.14, train accuracy = 92.19%, train recall = 100.00%, train precision = 75.76%
Step 98650, train loss = 0.09, train accuracy = 96.88%, train recall = 100.00%, train precision = 80.65%
Step 98700, train loss = 0.16, train accuracy = 95.31%,

Step 102150, train loss = 0.13, train accuracy = 95.31%, train recall = 100.00%, train precision = 72.73%
Step 102200, train loss = 0.23, train accuracy = 90.62%, train recall = 100.00%, train precision = 57.58%
Step 102250, train loss = 0.17, train accuracy = 92.19%, train recall = 100.00%, train precision = 76.47%
Step 102300, train loss = 0.21, train accuracy = 93.75%, train recall = 100.00%, train precision = 74.19%
Step 102350, train loss = 0.25, train accuracy = 90.62%, train recall = 100.00%, train precision = 70.59%
Step 102400, train loss = 0.14, train accuracy = 92.19%, train recall = 100.00%, train precision = 88.89%
Step 102450, train loss = 0.12, train accuracy = 95.31%, train recall = 100.00%, train precision = 78.38%
Step 102500, train loss = 0.26, train accuracy = 89.06%, train recall = 96.43%, train precision = 67.50%
Step 102550, train loss = 0.12, train accuracy = 96.88%, train recall = 100.00%, train precision = 73.53%
Step 102600, train loss = 0.17, train accuracy 